In [185]:
import os
from PIL import Image
from albumentations.pytorch import ToTensorV2
import albumentations as A
from torch import nn
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset
import numpy as np
import glob
import cv2
import imageio
import pandas as pd
import os
from torch.utils.data import DataLoader
import torch
import torchvision.transforms as T
from torchvision.models import resnet18, resnet50
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output
import warnings

from torch.optim import Adam
from sklearn.model_selection import train_test_split

In [186]:
class NatureDataset(Dataset):

    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        pic = self.df.iloc[index]["pics"]
        ans = self.df.iloc[index]["answers"]
        # print(pic)
        pic = np.array(Image.open(pic).convert("RGB"))
        if self.transform is not None:
            augmentations = self.transform(image = pic)
            pic = augmentations

        return pic["image"], ans

In [187]:
df = pd.DataFrame()
# кликун - 0, малой -1 , шипун - 2
path1 = r"C:\Users\ga232\Природа\root\klikun\images"
path2 = r"C:\Users\ga232\Природа\root\Maliy\images"
path3 = r"C:\Users\ga232\Природа\root\Shipun\images"


df["pic"] = os.listdir(path1)[:3000] + os.listdir(path2)[:3000] + os.listdir(path3)[:3000]
answers = np.append(np.array([0 for i in range(3000)]),np.array([1 for i in range(3000)]))
answers = np.append(answers,np.array([2 for i in range(3000)]))
df["answers"] = answers

#Справа тоже впиши свой путь
df["pic"].iloc[:3000] = df["pic"].iloc[:3000].apply(lambda x: f"C://Users//ga232//Природа//root//klikun//images//{x}" )
df["pic"].iloc[3000:6000] = df["pic"].iloc[3000:6000].apply(lambda x: f"C://Users//ga232//Природа//root//Maliy//images//{x}" )
df["pic"].iloc[6000:] = df["pic"].iloc[6000:].apply(lambda x: f"C://Users//ga232//Природа//root//Shipun//images//{x}" )

C:\Users\ga232\AppData\Local\Temp\ipykernel_19600\1901198503.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["pic"].iloc[:3000] = df["pic"].iloc[:3000].apply(lambda x: f"C://Users//ga232//Природа//root//klikun//images//{x}" )
C:\Users\ga232\AppData\Local\Temp\ipykernel_19600\1901198503.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["pic"].iloc[3000:6000] = df["pic"].iloc[3000:6000].apply(lambda x: f"C://Users//ga232//Природа//root//Maliy//images//{x}" )
C:\Users\ga232\AppData\Local\Temp\ipykernel_19600\1901198503.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [188]:
path_test = r"C:\Users\ga232\Природа\Тестовый датасет"
df_test = pd.DataFrame()
df_test["pics"] = os.listdir(path_test)
df_test["pics"] = df_test["pics"].apply(lambda x: f"C://Users//ga232//Природа//Тестовый датасет//{x}" )

In [189]:
df

,pic,answers
0,C://Users//ga232//Природа//root//klikun//image...,0
1,C://Users//ga232//Природа//root//klikun//image...,0
2,C://Users//ga232//Природа//root//klikun//image...,0
3,C://Users//ga232//Природа//root//klikun//image...,0
4,C://Users//ga232//Природа//root//klikun//image...,0
...,...,...
8995,C://Users//ga232//Природа//root//Shipun//image...,2
8996,C://Users//ga232//Природа//root//Shipun//image...,2
8997,C://Users//ga232//Природа//root//Shipun//image...,2
8998,C://Users//ga232//Природа//root//Shipun//image...,2


In [190]:
df_test

,pics
0,C://Users//ga232//Природа//Тестовый датасет//0...
1,C://Users//ga232//Природа//Тестовый датасет//1...
2,C://Users//ga232//Природа//Тестовый датасет//1...
3,C://Users//ga232//Природа//Тестовый датасет//1...
4,C://Users//ga232//Природа//Тестовый датасет//1...
...,...
590,C://Users//ga232//Природа//Тестовый датасет//9...
591,C://Users//ga232//Природа//Тестовый датасет//9...
592,C://Users//ga232//Природа//Тестовый датасет//9...
593,C://Users//ga232//Природа//Тестовый датасет//9...


In [191]:
#train, test = train_test_split(df, test_size=0.2)
test = df_test
train = df # обучался на всём датасете
train_transform = A.Compose(
    [

        A.Resize(224, 224),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        ToTensorV2(),
    ]
)

test_transform = A.Compose(
    [
        A.Resize(224, 224),
        ToTensorV2(),

    ]
)

In [192]:
df_test['answers'] = 0

In [193]:
df_test

,pics,answers
0,C://Users//ga232//Природа//Тестовый датасет//0...,0
1,C://Users//ga232//Природа//Тестовый датасет//1...,0
2,C://Users//ga232//Природа//Тестовый датасет//1...,0
3,C://Users//ga232//Природа//Тестовый датасет//1...,0
4,C://Users//ga232//Природа//Тестовый датасет//1...,0
...,...,...
590,C://Users//ga232//Природа//Тестовый датасет//9...,0
591,C://Users//ga232//Природа//Тестовый датасет//9...,0
592,C://Users//ga232//Природа//Тестовый датасет//9...,0
593,C://Users//ga232//Природа//Тестовый датасет//9...,0


In [194]:
train_dataset = NatureDataset(train, transform=train_transform)
test_dataset = NatureDataset(df_test["pics"], transform=test_transform)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True)
test_loader = DataLoader(test, batch_size=64, shuffle=True, pin_memory=True)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [195]:
next(iter(test_loader))

KeyError: 128

In [134]:
pic

,pics
0,C://Users//ga232//Природа//Тестовый датасет//0...
1,C://Users//ga232//Природа//Тестовый датасет//1...
2,C://Users//ga232//Природа//Тестовый датасет//1...
3,C://Users//ga232//Природа//Тестовый датасет//1...
4,C://Users//ga232//Природа//Тестовый датасет//1...
...,...
590,C://Users//ga232//Природа//Тестовый датасет//9...
591,C://Users//ga232//Природа//Тестовый датасет//9...
592,C://Users//ga232//Природа//Тестовый датасет//9...
593,C://Users//ga232//Природа//Тестовый датасет//9...


In [135]:
model.to("cpu")
torch.max(model(pic.type(torch.float32).unsqueeze(0)),1)[1]

AttributeError: 'DataFrame' object has no attribute 'type'

In [136]:
np.array(pic['pics'])

array(['C://Users//ga232//Природа//Тестовый датасет//0.jpg',
       'C://Users//ga232//Природа//Тестовый датасет//1.jpg',
       'C://Users//ga232//Природа//Тестовый датасет//10.jpg',
       'C://Users//ga232//Природа//Тестовый датасет//100.jpg',
       'C://Users//ga232//Природа//Тестовый датасет//101.jpg',
       'C://Users//ga232//Природа//Тестовый датасет//102.jpg',
       'C://Users//ga232//Природа//Тестовый датасет//103.jpg',
       'C://Users//ga232//Природа//Тестовый датасет//104.jpg',
       'C://Users//ga232//Природа//Тестовый датасет//105.jpg',
       'C://Users//ga232//Природа//Тестовый датасет//106.jpg',
       'C://Users//ga232//Природа//Тестовый датасет//107.jpg',
       'C://Users//ga232//Природа//Тестовый датасет//108.jpg',
       'C://Users//ga232//Природа//Тестовый датасет//109.jpg',
       'C://Users//ga232//Природа//Тестовый датасет//11.jpg',
       'C://Users//ga232//Природа//Тестовый датасет//110.jpg',
       'C://Users//ga232//Природа//Тестовый датасет//111.jpg'

In [206]:
test_transform = A.Compose(
    [
        A.Resize(224, 224),
        ToTensorV2(),

    ]
)

img = df_test

pic = df_test
#ans = img["answers"]
# print(pic)
pic = []

,pics,answers
0,C://Users//ga232//Природа//Тестовый датасет//0...,0
1,C://Users//ga232//Природа//Тестовый датасет//1...,0
2,C://Users//ga232//Природа//Тестовый датасет//1...,0
3,C://Users//ga232//Природа//Тестовый датасет//1...,0
4,C://Users//ga232//Природа//Тестовый датасет//1...,0
...,...,...
590,C://Users//ga232//Природа//Тестовый датасет//9...,0
591,C://Users//ga232//Природа//Тестовый датасет//9...,0
592,C://Users//ga232//Природа//Тестовый датасет//9...,0
593,C://Users//ga232//Природа//Тестовый датасет//9...,0


In [200]:

for x in df_test['pics']:
    pic = np.array(Image.open(x).convert("RGB"))
    if test_transform is not None:
        augmentations = test_transform(image = pic)
        pic = augmentations['image'].to("cpu")
    

In [205]:
pic

tensor([[[ 85,  83,  81,  ...,  81,  82,  85],
         [ 84,  82,  80,  ...,  81,  86,  84],
         [ 92,  85,  86,  ...,  89,  89,  91],
         ...,
         [ 82,  78,  82,  ..., 100,  97,  91],
         [ 85,  76,  87,  ...,  97, 100,  88],
         [ 82,  79,  87,  ...,  92, 100,  87]],

        [[ 95,  93,  91,  ...,  87,  89,  92],
         [ 94,  92,  90,  ...,  86,  93,  91],
         [102,  95,  96,  ...,  99,  96,  98],
         ...,
         [111, 107, 110,  ..., 121, 121, 114],
         [113, 104, 115,  ..., 118, 123, 111],
         [111, 107, 115,  ..., 115, 124, 112]],

        [[ 86,  84,  82,  ...,  85,  82,  85],
         [ 85,  83,  81,  ...,  83,  86,  84],
         [ 91,  84,  85,  ...,  90,  88,  90],
         ...,
         [ 45,  41,  49,  ...,  66,  69,  60],
         [ 48,  41,  55,  ...,  63,  71,  57],
         [ 47,  46,  56,  ...,  59,  72,  57]]], dtype=torch.uint8)

In [119]:
x,y = next(iter(test_loader))

KeyError: 484

In [138]:
model = resnet50(pretrained=True)



model.fc = nn.Linear(in_features=2048, out_features=3, bias=True)

model = model.to(device)

model.load_state_dict(torch.load("Final_model.pt"))
model.eval()
#print(evaluate(model1, test_loader))


C:\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [139]:
test

,pics
0,C://Users//ga232//Природа//Тестовый датасет//0...
1,C://Users//ga232//Природа//Тестовый датасет//1...
2,C://Users//ga232//Природа//Тестовый датасет//1...
3,C://Users//ga232//Природа//Тестовый датасет//1...
4,C://Users//ga232//Природа//Тестовый датасет//1...
...,...
590,C://Users//ga232//Природа//Тестовый датасет//9...
591,C://Users//ga232//Природа//Тестовый датасет//9...
592,C://Users//ga232//Природа//Тестовый датасет//9...
593,C://Users//ga232//Природа//Тестовый датасет//9...


In [140]:
x.shape

torch.Size([64, 3, 224, 224])

In [141]:
pred = torch.max(model(x.type(torch.float32).to(device)),1)[1]

In [142]:
sum(pred == y.to(device))/len(y)

tensor(1., device='cuda:0')

In [143]:
loss_fn = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0,3.0]).to(device))

In [144]:
@torch.inference_mode()
def evaluate(model, loader) -> tuple[float, float]:
    model.eval()

    total_loss = 0
    total = 0
    correct = 0

    for x, y in tqdm(loader, desc='Evaluation'):
        x, y = x.float().to(device), y.type(torch.LongTensor).to(device)

        output = model(x)

        loss = loss_fn(output, y)

        total_loss += loss.item()

        _, y_pred = torch.max(output, 1)
        total += y.size(0)
        correct += (y_pred == y).sum().item()

    total_loss /= len(loader)
    accuracy = correct / total

    return total_loss, accuracy

In [145]:
test_loader


In [155]:
test_loader

In [158]:
next(iter(test_loader))

KeyError: 145

In [156]:
for x in test_loader:
    print(x)
    break

KeyError: 288

In [ ]:
model